In [18]:
import sys
import os
import netCDF4
import xarray as xr
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import glob
import json
np.set_printoptions(threshold=np.inf)

In [19]:
places = 'data/places.json'

In [20]:
land_cover_data = []

In [21]:
with open(places, 'r') as places_file:
    places_data = json.load(places_file)

In [22]:
def get_landcover_data(ds, city, year, min_lat, max_lat, min_lon, max_lon):

    filtered_ds = ds.sel(lat=slice(max_lat, min_lat), lon=slice(min_lon, max_lon))

    flag_values = filtered_ds['lccs_class'].attrs['flag_values']
    flag_meanings = filtered_ds['lccs_class'].attrs['flag_meanings'].split()

    flag_dict = dict(zip(flag_values, flag_meanings))

    df = filtered_ds.to_dataframe()

    df['lccs_class'] = df['lccs_class'].replace(flag_dict)
    value_counts = df['lccs_class'].value_counts(normalize=True) * 100
    value_counts = value_counts.round(decimals=3)

    value_counts_dict = value_counts.to_dict()

    # loop through value_counts_dict
    for key, value in value_counts_dict.items():
        land_cover_data.append({
            'city': city,
            'year': year,
            'latutiude': min_lat + (max_lat - min_lat) / 2,
            'longitude': min_lon + (max_lon - min_lon) / 2,
            'land_cover_class': key,
            'percent': value
        })


In [23]:
def get_places(ds, year):
    for index, place in enumerate(places_data):
        if index < 3:
            print(place['city'])
            get_landcover_data(ds, place['city'], year, place['lat']-0.5, place['lat']+0.5, place['lon']-0.5, place['lon']+0.5)

In [24]:
for year in range(2010, 2019):
    print(year)
    if year >= 2016:
        ds = xr.open_dataset('data\C3S-LC-L4-LCCS-Map-300m-P1Y-' + str(year) + '-v2.1.1.nc')
    else:
        ds = xr.open_dataset('data\ESACCI-LC-L4-LCCS-Map-300m-P1Y-' + str(year) + '-v2.0.7cds.nc')
    get_places(ds, year)

2010
Cairo
Kinshasa
Lagos
2011
Cairo
Kinshasa
Lagos
2012
Cairo
Kinshasa
Lagos
2013
Cairo
Kinshasa
Lagos
2014
Cairo
Kinshasa
Lagos
2015
Cairo
Kinshasa
Lagos
2016
Cairo
Kinshasa
Lagos
2017
Cairo
Kinshasa
Lagos
2018
Cairo
Kinshasa
Lagos


In [25]:
lcdf = pd.DataFrame(land_cover_data)
lcdf.head()

,city,year,latutiude,longitude,land_cover_class,percent
0,Cairo,2010,30.04442,31.235712,200,54.268
1,Cairo,2010,30.04442,31.235712,20,30.067
2,Cairo,2010,30.04442,31.235712,190,8.191
3,Cairo,2010,30.04442,31.235712,11,4.950
4,Cairo,2010,30.04442,31.235712,10,0.738


In [26]:

# lcdf.to_csv('land_cover.csv', index=False)